<a href="https://colab.research.google.com/github/fd4dg4br1/Pesquisa-SUS/blob/main/SUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysus

In [ ]:
from google.colab import drive
import os


drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
RAW_DIR = os.path.join(BASE_DIR, 'data/raw')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

if os.path.exists(RAW_DIR):
    print("✅ Conexão bem sucedida!")
    print(f"Arquivos encontrados na pasta RAW: {os.listdir(RAW_DIR)}")
else:
    print(f"❌ Pasta não encontrada: {RAW_DIR}")
    print("Verifique se criou as pastas no Google Drive exatamente com esse nome.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Conexão bem sucedida!
Arquivos encontrados na pasta RAW: ['estimativa_dou_2021.xls', 'estimativa_dou_2014.xls', 'estimativa_dou_2015.xls', 'estimativa_TCU_2016_20170614.xls', 'estimativa_dou_2017.xls', 'estimativa_dou_2018_20181019.xls', 'estimativa_dou_2019.xls', 'estimativa_dou_2020.xls', 'estimativa_dou_2024.xls', 'estimativa_dou_2025.xls', 'sinan', '.ipynb_checkpoints', 'SINESP']


# SINAN

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
from pysus import SINAN, SIM, SIH # Nova API do PySUS
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# Configuração de Caminhos no Drive
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
RAW_DIR = os.path.join(BASE_DIR, 'data/raw/sinan')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# SINAN usa 6 dígitos. IBGE usa 7.
COD_MUNICIPIO_SINAN = "530010" # Brasília (6 dígitos)


COLUNAS_INTERESSE = {
    'DT_NOTIFIC': 'data_notificacao',
    'DT_OCOR': 'data_ocorrencia',
    'ID_MN_RESI': 'cod_ibge_residencia', # Onde a vítima mora
    'ID_MN_NOTI': 'cod_ibge_notificacao', # Onde a violência foi registrada
    'NU_IDADE_N': 'idade_codificada',
    'CS_SEXO': 'sexo',
    'CS_RACA': 'raca_cor',
    'CS_ESCOL_N': 'escolaridade',
    'SG_UF_NOTI': 'uf_notificacao',
    'ID_AGRAVO': 'id_agravo'
}

# Colunas booleanas de tipos de violência
VIOL_COLS_MAP = {
    'VIOL_FISIC': 'Física',
    'VIOL_PSICO': 'Psicológica',
    'VIOL_SEXU': 'Sexual',
    'VIOL_NEGLI': 'Negligência',
    'VIOL_FINAN': 'Financeira',
    'VIOL_TORT': 'Tortura',
    'LES_AUTOP': 'Autoprovocada' # Tentativa de suicídio entra aqui
}


def decodificar_idade(valor_codificado):
    """
    O SINAN codifica idade assim:
    4021 -> 4 (anos) + 021 (anos de idade) = 21 anos
    3005 -> 3 (meses) + 005 (meses) = Bebê de 5 meses -> 0 anos
    """
    try:
        valor_str = str(int(valor_codificado))
        if len(valor_str) < 2: return np.nan

        unidade = valor_str[0] # 4=anos, 3=meses, 2=dias, 1=horas
        valor = int(valor_str[1:])

        if unidade == '4':
            return valor
        elif unidade in ['1', '2', '3']:
            return 0 # Menor de 1 ano
        else:
            return np.nan # Ignorado/Erro
    except:
        return np.nan

def consolidar_violencia(row):
    """Cria uma lista legível dos tipos de violência sofrida"""
    tipos = []
    for col_codigo, nome_legivel in VIOL_COLS_MAP.items():
        if col_codigo in row:
            val = str(row[col_codigo]).upper().strip()
            # SINAN usa '1' ou 'S' para Sim
            if val in ['1', 'S', 'TRUE', '1.0']:
                tipos.append(nome_legivel)

    if not tipos:
        return "Não Especificado"
    return ", ".join(tipos)

def processar_arquivo_parquet(caminho_arquivo):
    """
    Lê um parquet bruto baixado pelo PySUS, filtra DF e limpa.
    """
    try:

        df_schema = pd.read_parquet(caminho_arquivo, columns=[])
        cols_presentes = list(df_schema.columns) # workaround simples ou ler tudo

        # Lê o arquivo completo
        df = pd.read_parquet(caminho_arquivo)

        # 2. Filtro Geográfico
        if 'ID_MN_RESI' in df.columns:
            df['ID_MN_RESI'] = df['ID_MN_RESI'].astype(str).str.replace(r'\.0$', '', regex=True)

            # Filtra: Ou mora no DF (530010) OU a notificação foi no DF (530010)
            mask_df = (df['ID_MN_RESI'] == COD_MUNICIPIO_SINAN)
            df = df[mask_df].copy()
        else:
            return None # Arquivo sem coluna de município

        if df.empty:
            return None

        # 3. Seleção e Renomeação
        cols_para_manter = [c for c in COLUNAS_INTERESSE.keys() if c in df.columns]
        cols_violencia = [c for c in VIOL_COLS_MAP.keys() if c in df.columns]

        df_clean = df[cols_para_manter + cols_violencia].copy()
        df_clean.rename(columns=COLUNAS_INTERESSE, inplace=True)

        # 4. Engenharia de Features
        # Decodificar Idade
        if 'idade_codificada' in df_clean.columns:
            df_clean['idade_real'] = df_clean['idade_codificada'].apply(decodificar_idade)

        # Consolidar Tipos de Violência
        df_clean['tipo_violencia'] = df_clean.apply(consolidar_violencia, axis=1)

        # Datas
        for col_data in ['data_notificacao', 'data_ocorrencia']:
            if col_data in df_clean.columns:
                df_clean[col_data] = pd.to_datetime(df_clean[col_data], errors='coerce')

        # Remove colunas brutas de violência para economizar espaço
        df_clean.drop(columns=[c for c in cols_violencia if c in df_clean.columns], inplace=True)

        return df_clean

    except Exception as e:
        print(f"Erro ao processar {os.path.basename(caminho_arquivo)}: {e}")
        return None


def executar_pipeline_sinan():
    print("🚀 Iniciando Pipeline SINAN - Violência (2014-2025)...")

    # 1. Carregar Metadata
    try:
        sinan = SINAN().load()
    except:
        sinan = SINAN().load()

    years = list(range(2014, 2026))

    print("📥 Consultando metadados...")
    files_metadata = sinan.get_files(dis_code="VIOL", year=years)

    if not files_metadata:
        print("❌ Nenhum arquivo encontrado.")
        return

    # 2. Download
    print(f"📥 Baixando arquivos...")
    try:
        # Retorna uma lista de objetos ParquetSet
        resultados_download = sinan.download(files_metadata)
    except Exception as e:
        print(f"❌ Erro no download: {e}")
        return

    # --- CORREÇÃO: USANDO O ATRIBUTO .path ---
    caminhos_reais = []

    # Normaliza para lista se vier um objeto solto
    if not isinstance(resultados_download, list):
        resultados_download = [resultados_download]

    print("📦 Extraindo caminhos (Via atributo .path)...")

    for item in resultados_download:
        if hasattr(item, 'path'):
            caminho = item.path
            # As vezes o path pode vir como string ou lista de strings
            if isinstance(caminho, str):
                caminhos_reais.append(caminho)
            elif isinstance(caminho, list):
                caminhos_reais.extend(caminho)
            else:
                # Caso venha como objeto PosixPath
                caminhos_reais.append(str(caminho))

        # Fallback para string direta
        elif isinstance(item, str):
            caminhos_reais.append(item)

    # Remove duplicatas e strings vazias
    caminhos_reais = list(set([str(c) for c in caminhos_reais if c]))

    print(f"✅ Arquivos extraídos: {len(caminhos_reais)}")

    if len(caminhos_reais) == 0:
        print("❌ CRÍTICO: Falha na extração. Verifique se os downloads ocorreram.")
        return

    # 3. Processamento Paralelo
    print(f"⚙️ Processando {len(caminhos_reais)} arquivos locais...")

    dfs_processados = []

    with ThreadPoolExecutor(max_workers=4) as executor:
        # Mapeia a função de processamento para cada arquivo
        resultados = list(tqdm(executor.map(processar_arquivo_parquet, caminhos_reais), total=len(caminhos_reais)))

    # Filtra None
    dfs_processados = [df for df in resultados if df is not None]

    # 4. Consolidação Final
    if dfs_processados:
        print("📊 Consolidando DataFrames...")
        df_final = pd.concat(dfs_processados, ignore_index=True)

        if 'data_notificacao' in df_final.columns:
            df_final.sort_values('data_notificacao', inplace=True)

        # Salvar
        caminho_saida = os.path.join(PROCESSED_DIR, 'sinan_violencia_df_consolidado.parquet')
        caminho_saida_csv = os.path.join(PROCESSED_DIR, 'sinan_violencia_df_consolidado.csv')

        df_final.to_parquet(caminho_saida, index=False)
        df_final.to_csv(caminho_saida_csv, index=False)

        print(f"\n✅ SUCESSO! Base consolidada salva em:")
        print(f"   📄 {caminho_saida_csv}")
        print(f"   Registros Totais: {len(df_final)}")
        print("\n🔎 Amostra dos Dados:")
        print(df_final[['data_notificacao', 'sexo', 'idade_real', 'tipo_violencia']].head())

        return df_final
    else:
        print("⚠️ Processamento concluído. Nenhum registro atendeu aos filtros (DF 530010).")
#------------------------------------------------------------
# EXECUÇÃO
# -----------------------------------------------------------
if __name__ == "__main__":
    df_sinan = executar_pipeline_sinan()

🚀 Iniciando Pipeline SINAN - Violência (2014-2025)...
📥 Consultando metadados...
📥 Baixando arquivos...


VIOLBR17.parquet:  27%|██▋       | 240k/882k [00:42<02:03, 5.21kB/s]

KeyboardInterrupt: 

# SIM

In [ ]:
import os
import pandas as pd
import numpy as np
from pysus import SIM
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

# Garante que o diretório existe
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Mapeamento de Colunas SIM -> Painel
COLS_SIM = {
    'DTOBITO': 'data_obito',
    'DTNASC': 'data_nascimento',
    'CODMUNRES': 'cod_ibge_residencia',
    'CODMUNOCOR': 'cod_ibge_ocorrencia',
    'CAUSABAS': 'causa_basica_cid10',
    'SEXO': 'sexo',
    'RACACOR': 'raca_cor',
    'LOCOCOR': 'local_ocorrencia',
    'CIRCOBIT': 'circunstancia_obito'
}

# --- FUNÇÕES ---

def filtrar_causas_externas(df):

    if 'causa_basica_cid10' not in df.columns: return df

    # Pega primeira letra maiúscula
    capitulo = df['causa_basica_cid10'].astype(str).str.upper().str.strip().str[0]

    return df[capitulo.isin(['V', 'W', 'X', 'Y', 'F'])].copy()

def processar_sim_parquet(caminho_arquivo):
    try:
        # Leitura
        df = pd.read_parquet(caminho_arquivo)

        cols_existentes = [c for c in COLS_SIM.keys() if c in df.columns]
        df_clean = df[cols_existentes].copy()
        df_clean.rename(columns=COLS_SIM, inplace=True)

        df_clean = filtrar_causas_externas(df_clean)

        if df_clean.empty: return None

        # 3. Tratamento de Datas
        for col in ['data_obito', 'data_nascimento']:
            if col in df_clean.columns:
                # SIM costuma ser DDMMAAAA
                df_clean[col] = pd.to_datetime(df_clean[col], format='%d%m%Y', errors='coerce')

        # 4. Cálculo de Idade (CORREÇÃO DO ERRO 'Unit Y')
        if 'data_obito' in df_clean.columns and 'data_nascimento' in df_clean.columns:
            # Calculamos a diferença em dias ('D') e dividimos por 365.25
            diferenca_dias = (df_clean['data_obito'] - df_clean['data_nascimento']) / np.timedelta64(1, 'D')
            df_clean['idade_anos'] = diferenca_dias / 365.25
            df_clean['idade_anos'] = df_clean['idade_anos'].round(1)

        return df_clean

    except Exception as e:
        print(f"Erro ao processar arquivo: {e}")
        return None

# --- ORQUESTRADOR ---

def executar_pipeline_sim_df():
    print("🚀 Iniciando Pipeline SIM (Mortalidade) - Foco DF...")

    try:
        sim = SIM().load()
    except:
        sim = SIM().load()

    years = list(range(2014, 2026))

    print("📥 Consultando arquivos do DF...")
    try:
        files_metadata = sim.get_files("CID10", uf="DF", year=years)
    except Exception as e:
        print(f"Erro ao buscar arquivos: {e}")
        return

    if not files_metadata:
        print("❌ Nenhum arquivo encontrado para DF nesses anos.")
        return

    print(f"📥 Baixando {len(files_metadata)} arquivos...")
    try:
        res_download = sim.download(files_metadata)
    except Exception as e:
        print(f"❌ Erro download: {e}")
        return

    caminhos = []
    if not isinstance(res_download, list): res_download = [res_download]

    for item in res_download:
        if hasattr(item, 'path'):
            p = item.path
            if isinstance(p, list): caminhos.extend(p)
            else: caminhos.append(str(p))
        elif isinstance(item, str):
            caminhos.append(item)

    caminhos = list(set([str(c) for c in caminhos if c]))
    print(f"✅ Arquivos extraídos: {len(caminhos)}")

    # Processamento
    print(f"⚙️ Processando...")
    with ThreadPoolExecutor(max_workers=4) as executor:
        resultados = list(tqdm(executor.map(processar_sim_parquet, caminhos), total=len(caminhos)))

    dfs = [df for df in resultados if df is not None]

    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)

        if 'data_obito' in df_final.columns:
            df_final.sort_values('data_obito', inplace=True)

        # Salva CSV
        path_csv = os.path.join(PROCESSED_DIR, 'sim_mortalidade_externa_df.csv')
        df_final.to_csv(path_csv, index=False)

        # Salva Parquet
        path_pq = os.path.join(PROCESSED_DIR, 'sim_mortalidade_externa_df.parquet')
        df_final.to_parquet(path_pq, index=False)

        print(f"\n✅ SUCESSO! Base SIM gerada.")
        print(f"   Total de Óbitos por Causas Externas: {len(df_final)}")
        print(f"   Arquivos salvos em: {PROCESSED_DIR}")
        print("   Amostra:")
        print(df_final[['data_obito', 'causa_basica_cid10', 'sexo']].head())
    else:
        print("❌ Nenhum dado restou após filtragem.")

if __name__ == "__main__":
    executar_pipeline_sim_df()

🚀 Iniciando Pipeline SIM (Mortalidade) - Foco DF...
📥 Consultando arquivos do DF...
📥 Baixando 11 arquivos...


DODF2024.parquet: 100%|██████████| 38.9k/38.9k [00:02<00:00, 13.3kB/s]

✅ Arquivos extraídos: 11
⚙️ Processando...


  0%|          | 0/11 [00:00<?, ?it/s]


✅ SUCESSO! Base SIM gerada.
   Total de Óbitos por Causas Externas: 20829
   Arquivos salvos em: /content/drive/MyDrive/criminalidade_df/data/processed
   Amostra:
      data_obito causa_basica_cid10  sexo
15003 2014-01-01           X954      1   
15002 2014-01-01           X999      1   
15001 2014-01-01           Y294      2   
15000 2014-01-01           F102      1   
14999 2014-01-01           W190      1   


# SIH

In [ ]:
import os
import pandas as pd
import numpy as np
from pysus import SIH
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')
COD_MUNICIPIO_RESIDENCIA = "530010" # Brasília (6 dígitos)

# Mapeamento SIH (AIH Reduzida) -> Painel
# Nomes das colunas variam no SIH, mas geralmente seguem esse padrão
COLS_SIH = {
    'DT_INTER': 'data_internacao',
    'DT_SAIDA': 'data_saida',
    'MUNIC_RES': 'cod_ibge_residencia',
    'DIAG_PRINC': 'diag_principal_cid10', # Motivo principal da internação
    'DIAG_SECUN': 'diag_secundario_cid10', # Motivo da causa externa (ex: tiro) muitas vezes está aqui
    'VAL_TOT': 'valor_total',
    'SEXO': 'sexo',
    'UTI_MES_TO': 'dias_uti',
    'MORTE': 'obito_internacao', # 1=Sim, 0=Não
    'COBRANCA': 'motivo_saida'
}

# --- FUNÇÕES ---

codigos_interesse = ('V', 'W', 'X', 'Y', 'S', 'T', 'F')

def filtrar_causas_externas_sih(df):
    """
    No SIH, a violência pode aparecer no Diagnóstico Principal OU Secundário.
    Filtramos se QUALQUER UM deles começar com V, W, X, Y (Capítulo XX).
    """
    mask_final = pd.Series(False, index=df.index)

    # Verifica Diagnóstico Principal
    if 'diag_principal_cid10' in df.columns:
        s_princ = df['diag_principal_cid10'].astype(str).str.upper()
        mask_final = mask_final | s_princ.str.startswith(codigos_interesse)

    # Verifica Diagnóstico Secundário (MUITO IMPORTANTE para traumas)
    if 'diag_secundario_cid10' in df.columns:
        s_sec = df['diag_secundario_cid10'].astype(str).str.upper()
        mask_final = mask_final | s_sec.str.startswith(codigos_interesse)

    return df[mask_final].copy()

def processar_sih_parquet(caminho_arquivo):
    try:
        df = pd.read_parquet(caminho_arquivo)

        # 1. Filtro Geográfico (Residentes no DF)
        if 'MUNIC_RES' in df.columns:
            df['MUNIC_RES'] = df['MUNIC_RES'].astype(str).str.replace(r'\.0$', '', regex=True)
            df = df[df['MUNIC_RES'] == COD_MUNICIPIO_RESIDENCIA].copy()

        if df.empty: return None

        # 2. Seleção e Renomeação
        cols_existentes = [c for c in COLS_SIH.keys() if c in df.columns]
        df_clean = df[cols_existentes].copy()
        df_clean.rename(columns=COLS_SIH, inplace=True)

        # 3. Filtro de Causa (Apenas Violência/Acidentes)
        df_clean = filtrar_causas_externas_sih(df_clean)

        if df_clean.empty: return None

        # 4. Tratamento de Datas
        for col in ['data_internacao', 'data_saida']:
            if col in df_clean.columns:
                df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

        return df_clean

    except Exception as e:
        # SIH as vezes tem arquivos corrompidos antigos, apenas ignoramos
        # print(f"Erro no arquivo {os.path.basename(caminho_arquivo)}: {e}")
        return None

# --- ORQUESTRADOR SIH ---

def executar_pipeline_sih():
    print("🚀 Iniciando Pipeline SIH - Internações (2014-2025)...")

    try:
        sih = SIH().load()
    except:
        sih = SIH().load()

    years = list(range(2014, 2026))
    months = list(range(1, 13)) # SIH precisa de meses 1 a 12

    print("📥 Consultando metadados (RD - AIH Reduzida)...")
    # AQUI ESTÁ A MUDANÇA DA DOCUMENTAÇÃO: passamos months e dis_code='RD'
    files_metadata = sih.get_files("RD", year=years, month=months, uf="DF")

    if not files_metadata:
        print("❌ Nenhum arquivo encontrado.")
        return

    print(f"📥 Baixando {len(files_metadata)} arquivos mensais...")
    try:
        resultados_download = sih.download(files_metadata)
    except Exception as e:
        print(f"❌ Erro no download: {e}")
        return

    # --- Extração "Blindada" de Caminhos ---
    caminhos_reais = []
    if not isinstance(resultados_download, list):
        resultados_download = [resultados_download]

    print("📦 Extraindo caminhos...")
    for item in resultados_download:
        if hasattr(item, 'path'):
            caminho = item.path
            if isinstance(caminho, str): caminhos_reais.append(caminho)
            elif isinstance(caminho, list): caminhos_reais.extend(caminho)
            else: caminhos_reais.append(str(caminho))
        elif isinstance(item, str):
            caminhos_reais.append(item)

    caminhos_reais = list(set([str(c) for c in caminhos_reais if c]))
    print(f"✅ Arquivos mensais extraídos: {len(caminhos_reais)}")

    if len(caminhos_reais) == 0:
        print("❌ Falha na extração.")
        return

    # Processamento em Lotes (Para economizar RAM do Colab)
    print(f"⚙️ Processando arquivos...")

    # Vamos processar e consolidar em memória
    dfs_validos = []

    # ThreadPool com menos workers pois arquivos SIH são maiores
    with ThreadPoolExecutor(max_workers=2) as executor:
        resultados = list(tqdm(executor.map(processar_sih_parquet, caminhos_reais), total=len(caminhos_reais)))

    dfs_validos = [df for df in resultados if df is not None]

    if dfs_validos:
        print("📊 Consolidando DataFrames...")
        df_final = pd.concat(dfs_validos, ignore_index=True)

        if 'data_internacao' in df_final.columns:
            df_final.sort_values('data_internacao', inplace=True)

        # Salva
        path_pq = os.path.join(PROCESSED_DIR, 'sih_internacoes_externas_df.parquet')
        path_csv = os.path.join(PROCESSED_DIR, 'sih_internacoes_externas_df.csv')

        df_final.to_parquet(path_pq, index=False)
        # O CSV do SIH pode ficar grande, cuidado ao abrir no Excel
        df_final.to_csv(path_csv, index=False)

        print(f"\n✅ SUCESSO! Base SIH gerada com {len(df_final)} internações por causas externas.")
        print("   Amostra:")
        print(df_final[['data_internacao', 'sexo', 'diag_principal_cid10', 'valor_total']].head())
    else:
        print("❌ Nenhum dado encontrado.")

if __name__ == "__main__":
    executar_pipeline_sih()

🚀 Iniciando Pipeline SIH - Internações (2014-2025)...
📥 Consultando metadados (RD - AIH Reduzida)...
📥 Baixando 143 arquivos mensais...


RDDF2511.parquet: 100%|██████████| 69.5k/69.5k [00:04<00:00, 15.1kB/s]

📦 Extraindo caminhos...
✅ Arquivos mensais extraídos: 143
⚙️ Processando arquivos...


  0%|          | 0/143 [00:00<?, ?it/s]

📊 Consolidando DataFrames...

✅ SUCESSO! Base SIH gerada com 178935 internações por causas externas.
   Amostra:
     data_internacao sexo diag_principal_cid10     valor_total
3666      2013-09-12    1                 S061          907.74
3665      2013-09-12    3                 S720         3957.63
3614      2013-09-20    3                 S920          600.60
3663      2013-09-22    1                 S024         1464.69
3619      2013-09-23    3                 S272         5378.26
